### Background

Let $\Pi$ be the function ```unitary_to_monodromy```. Then for $XX(\alpha) = \exp(-i\alpha XX)$, we have $\Pi(XX(\alpha)) = \left( \frac{\alpha}{\pi}, \frac{\alpha}{\pi}, -\frac{\alpha}{\pi}, -\frac{\alpha}{\pi}\right)$. Since $0 \leq \alpha \leq \frac{\pi}{4}$ by page $4$ of $XX$, we have that $0\leq\frac{\alpha}{\pi}=x_1 \leq \frac{1}{4}$.

### Computing XX Polytope

In [2]:
from time import perf_counter

import sys

sys.path.insert(1, "/Users/minhpham/Documents/Research/laughing-umbrella/xx_synthesis/monodromy/")


import monodromy

from monodromy.coordinates import monodromy_alcove, monodromy_alcove_c2, monodromy_to_positive_canonical_polytope, rho_reflect
from monodromy.elimination import cylinderize, project
from monodromy.polytopes import ConvexPolytope, Polytope
from monodromy.static import qlr_polytope

from itertools import count

biswas_relations = (qlr_polytope
    # enlarge to the pu_4 version of the QLR relations
    .union(rho_reflect(qlr_polytope, [0, 7, 8, 9]))
    # constrain in- and out-coordinates to the appropriate alcove
    .intersect(cylinderize(monodromy_alcove, [0, 1, 2, 3], 10))
    .intersect(cylinderize(monodromy_alcove_c2, [0, 7, 8, 9], 10))
)

# constrain interaction coordinates to be of XX-type
biswas_relations = biswas_relations.intersect(Polytope(convex_subpolytopes=[
    ConvexPolytope(
        inequalities=[[1, 0, 0, 0, -4, 0, 0, 0, 0, 0]],
        equalities=[
            [0, 0, 0, 0, 1, -1, 0, 0, 0, 0],  # x1 == x2
            [0, 0, 0, 0, 0, 1, 1, 0, 0, 0],  # x2 == -x3
        ]
    )
]))

# switch to canonical coordinates
biswas_relations = monodromy_to_positive_canonical_polytope(
    biswas_relations, coordinates=[0, 1, 2, 3])
biswas_relations = monodromy_to_positive_canonical_polytope(
    biswas_relations, coordinates=[0, 4, 5, 6])
biswas_relations = monodromy_to_positive_canonical_polytope(
    biswas_relations, coordinates=[0, 7, 8, 9])

# reduce the biswas relations to have following coordinates:
# k a1 a2 a3 beta b1 b2 b3
biswas_relations = biswas_relations.reduce()
biswas_relations = project(biswas_relations, 6).reduce()
biswas_relations = project(biswas_relations, 5).reduce()
    
xx_polytope = monodromy.static.examples.identity_polytope
polytope_list = []
for n in range(4):
    
    print(f"Working on an XX interaction sequence of length {n}...")
    
    # inflate xx_polytope from [*a_coords, *interaction_coords] to [*a_coords, *b_coords, *interaction_coords, beta]
    xx_polytope = cylinderize(
        xx_polytope,
        coordinate_map=[0, 1, 2, 3] + list(range(7, 7 + (n - 1))),
        parent_dimension=1 + 3 + 3 + n,
    ).intersect(cylinderize(
        biswas_relations,
        coordinate_map=[0, 1, 2, 3, -1, 4, 5, 6],
        parent_dimension=1 + 3 + 3 + n,
    ))
    
    # project away the old a-coordinates
    start_time = perf_counter()
    print("Working on the reduction 1/3...", end="")
    xx_polytope = project(xx_polytope, 3).reduce()
    print(f" done.  Took {perf_counter() - start_time} seconds.")

    start_time = perf_counter()
    print("Working on the reduction 2/3...", end="")
    xx_polytope = project(xx_polytope, 2).reduce()
    print(f" done.  Took {perf_counter() - start_time} seconds.")

    start_time = perf_counter()
    print("Working on the reduction 3/3...", end="")
    xx_polytope = project(xx_polytope, 1).reduce()
    print(f" done.  Took {perf_counter() - start_time} seconds.")
    
    # now the old c-coordinates are sitting where the a-coordinates were!
    print("The first three coordinates are the canonical coordinates CAN(x1, x2, x3).")
    print("The remaining coordinates x4, ..., xk are the XX interaction strengths.")
    print(xx_polytope)
    
    polytope_list.append(xx_polytope)

Working on an XX interaction sequence of length 0...
Working on the reduction 1/3... done.  Took 0.1176362729165703 seconds.
Working on the reduction 2/3... done.  Took 0.11319456505589187 seconds.
Working on the reduction 3/3... done.  Took 0.1360569349490106 seconds.
The first three coordinates are the canonical coordinates CAN(x1, x2, x3).
The remaining coordinates x4, ..., xk are the XX interaction strengths.
[
# origin ∩ anonymous_convex_polytope_15: 
    1 +     0 x1 +     0 x2 +     0 x3 >= 0
    0 +     1 x1 +     0 x2 +     0 x3 == 0
    0 +     0 x1 +     1 x2 +     0 x3 == 0
    0 +     0 x1 +     0 x2 +     1 x3 == 0
,
# origin ∩ anonymous_convex_polytope_16: 
    1 +     0 x1 +     0 x2 +     0 x3 >= 0
   -1 +     1 x1 +     0 x2 +     0 x3 == 0
    0 +     0 x1 +     1 x2 +     0 x3 == 0
    0 +     0 x1 +     0 x2 +     1 x3 == 0

]
Working on an XX interaction sequence of length 1...
Working on the reduction 1/3... done.  Took 0.24839976895600557 seconds.
Working on the

### Print Table

In [3]:
def print_table(data, no_extra_rows, name):
    # Define the headers
    headers = ["1", "a_1", "a_2", "a_3"] + [f"𝛼_{i+1}" for i in range(no_extra_rows)]
    
    data = sorted(data)
    
    # Ensure that the data length matches the number of headers
    if not all(len(row) == len(headers) for row in data):
        raise ValueError("All rows in the data must have the same length as the headers.")
    
    # Calculate the maximum width for each column
    col_widths = [max(len(str(item)) for item in [header] + [row[i] for row in data]) for i, header in enumerate(headers)]
    
    # Define a function to print a single row with given data and column widths
    def print_row(row):
        print(" | ".join(f"{str(item).ljust(col_widths[i])}" for i, item in enumerate(row)))
    
    # Print name
    print(f"{name} ({len(data)} inequalities)\n")
    
    # Print the headers
    print_row(headers)
    print("-+-".join("-" * width for width in col_widths))
    
    # Print each row of data
    for row in data:
        print_row(row)
        
    print("\n")

def classify(ineq_system):
    
    interaction_ineq, CAN_ineq, total_strength_ineq, slant_ineq, height_ineq, other_ineq = [], [], [], [], [], []
    for ineq in ineq_system:
        if ineq[1:4] == [0, 0, 0]:
            interaction_ineq.append(ineq)
        elif ineq[4:] == [0 for i in range(len(ineq[4:]))]:
            CAN_ineq.append(ineq)
        elif ineq[1:4] == [-1, -1, -1]:
            total_strength_ineq.append(ineq)
        elif ineq[1:4] == [1, -1, -1]:
            slant_ineq.append(ineq)
        elif ineq[1:4] == [0, 0, -1]:
            height_ineq.append(ineq)
        else:
            other_ineq.append(ineq)
    
    n = len(ineq)-4
    print_table(interaction_ineq, n, "Interaction Ineq")
    print_table(CAN_ineq, n, "CAN Ineq")
    print_table(total_strength_ineq, n, "Total Strength Ineq")
    print_table(slant_ineq, n, "Slant Ineq")
    print_table(height_ineq, n, "Height Ineq")
    print_table(other_ineq, n, "Other Ineq")
                
    return interaction_ineq, CAN_ineq, total_strength_ineq, slant_ineq, height_ineq, other_ineq

In [4]:
polytope = polytope_list[3].convex_subpolytopes[1].inequalities
interaction_ineq, CAN_ineq, total_strength_ineq, slant_ineq, height_ineq, other_ineq = classify(polytope)

Interaction Ineq (3 inequalities)

1 | a_1 | a_2 | a_3 | 𝛼_1 | 𝛼_2 | 𝛼_3
--+-----+-----+-----+-----+-----+----
1 | 0   | 0   | 0   | -2  | 0   | 0  
1 | 0   | 0   | 0   | 0   | -2  | 0  
1 | 0   | 0   | 0   | 0   | 0   | -2 


CAN Ineq (4 inequalities)

1 | a_1 | a_2 | a_3 | 𝛼_1 | 𝛼_2 | 𝛼_3
--+-----+-----+-----+-----+-----+----
0 | 0   | 0   | 1   | 0   | 0   | 0  
0 | 0   | 1   | -1  | 0   | 0   | 0  
0 | 1   | -1  | 0   | 0   | 0   | 0  
1 | -1  | -1  | 0   | 0   | 0   | 0  


Total Strength Ineq (3 inequalities)

1 | a_1 | a_2 | a_3 | 𝛼_1 | 𝛼_2 | 𝛼_3
--+-----+-----+-----+-----+-----+----
1 | -1  | -1  | -1  | -1  | 1   | 1  
1 | -1  | -1  | -1  | 1   | -1  | 1  
1 | -1  | -1  | -1  | 1   | 1   | -1 


Slant Ineq (1 inequalities)

1  | a_1 | a_2 | a_3 | 𝛼_1 | 𝛼_2 | 𝛼_3
---+-----+-----+-----+-----+-----+----
-1 | 1   | -1  | -1  | 1   | 1   | 1  


Height Ineq (3 inequalities)

1 | a_1 | a_2 | a_3 | 𝛼_1 | 𝛼_2 | 𝛼_3
--+-----+-----+-----+-----+-----+----
0 | 0   | 0   | -1  | 0   | 0   

### XX inequalities

#### Polytope 1

$\alpha_+ \geq a_1+ a_2 + a_3$
```python

Total Strength Ineq (1 inequalities)

1 | a_1 | a_2 | a_3 | 𝛼_1 | 𝛼_2 | 𝛼_3
--+-----+-----+-----+-----+-----+----
0 | -1  | -1  | -1  | 1   | 1   | 1  
```
$\alpha_+ - 2\alpha^\prime \geq -a_1 + a_2+a_3$
```python
Slant Ineq (3 inequalities)

1 | a_1 | a_2 | a_3 | 𝛼_1 | 𝛼_2 | 𝛼_3
--+-----+-----+-----+-----+-----+----
0 | 1   | -1  | -1  | -1  | 1   | 1  
0 | 1   | -1  | -1  | 1   | -1  | 1  
0 | 1   | -1  | -1  | 1   | 1   | -1 
```
$\alpha_+ - \alpha^\prime - \alpha^{\prime\prime} \geq a_3$
```python
Height Ineq (3 inequalities)

1 | a_1 | a_2 | a_3 | 𝛼_1 | 𝛼_2 | 𝛼_3
--+-----+-----+-----+-----+-----+----
0 | 0   | 0   | -1  | 0   | 0   | 1  
0 | 0   | 0   | -1  | 0   | 1   | 0  
0 | 0   | 0   | -1  | 1   | 0   | 0

```

#### Polytope 2
$-\frac{\pi}{2} + \alpha_+ \geq -a_1+ a_2 + a_3$
```python
Total Strength Ineq (3 inequalities)

1  | a_1 | a_2 | a_3 | 𝛼_1 | 𝛼_2 | 𝛼_3
---+-----+-----+-----+-----+-----+----
-1 | 1   | -1  | -1  | 1   | 1   | 1  

```
$\frac{\pi}{2} + \alpha_+ - 2\alpha^\prime  \geq a_1 + a_2+a_3$
```python
Slant Ineq (1 inequalities)

1 | a_1 | a_2 | a_3 | 𝛼_1 | 𝛼_2 | 𝛼_3
--+-----+-----+-----+-----+-----+----
1 | -1  | -1  | -1  | -1  | 1   | 1  
1 | -1  | -1  | -1  | 1   | -1  | 1  
1 | -1  | -1  | -1  | 1   | 1   | -1 
```
$\alpha_+ - \alpha^\prime - \alpha^{\prime\prime} \geq a_3$
```python
Height Ineq (3 inequalities)

1 | a_1 | a_2 | a_3 | 𝛼_1 | 𝛼_2 | 𝛼_3
--+-----+-----+-----+-----+-----+----
0 | 0   | 0   | -1  | 0   | 0   | 1  
0 | 0   | 0   | -1  | 0   | 1   | 0  
0 | 0   | 0   | -1  | 1   | 0   | 0 

```

SyntaxError: unexpected character after line continuation character (2899035242.py, line 1)